In [ ]:
%%caprure
!pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 50.1 MB/s eta 0:00:00


In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import plotly.express as px
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import pymorphy3
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Load Data

In [ ]:
DATA = pl.read_parquet('hf://datasets/d0rj/povarenok_recipes_detail/data/train-*-of-*.parquet',
                     columns=['title', 'description', 'kroshki', 'ingredients', 'tags', 'tastes', 'cooking_steps'])

DATA = DATA.to_pandas()

# Data Describtion

Датасет содержит различную информацию о рецептах с сайта "Поваренок".

В работе актуальны:

 ```['title' , 'description', 'kroshki', 'ingredients', 'tags', 'tastes']```

In [ ]:
print(DATA.shape)
DATA.head()

(154158, 6)


,title,description,kroshki,ingredients,tags,tastes
0,"Бутерброды ""Кораблики с икрой""",Как можно украсить обычные бутерброды с красно...,"[Закуски, Бутерброды, Открытый бутерброд]","[{'count': None, 'name': 'Батон'}, {'count': '...","[бутерброд, закуска]","[соленый, пикантный, кислый]"
1,Баклажаны с грибами под сметаной,Обжаренные баклажаны с жареными грибами в смет...,"[Горячие блюда, Запеканки, Запеканка с овощами]","[{'count': '500 г', 'name': 'Баклажан'}, {'cou...","[ужин, обед, горячее]","[грибной, баклажановый]"
2,Мясо в сухарях,"Отбитые кусочки мяса, обжаренные в панировочн...","[Горячие блюда, Блюда из мяса, Горячие блюда ...","[{'count': '800 г', 'name': 'Свинина'}, {'coun...","[обед, второе]","[пикантный, острый]"
3,Фаршированная щука,"Вообще это больше, банкетное блюдо, но я думаю...","[Горячие блюда, Блюда из рыбы и морепродуктов,...","[{'count': '1 шт', 'name': 'Щука'}, {'count': ...","[обед, ужин, праздник]",[вкусный]
4,Мясо в виноградных листьях,Фарш со вкусом виноградных листьев.,"[Горячие блюда, Блюда из мяса, Горячие блюда ...","[{'count': '500 г', 'name': 'Фарш мясной'}, {'...","[второе, фарш, долма]","[жареный, острый, соленый]"


## Задача 1: Рекомендательная система на основе текстовых данных

In [ ]:
df = DATA[['description', 'ingredients']].copy()

Проверим на наличие пропусков, как видно, они есть, но в небольшом количестве. Поэтому просто удалим.

In [ ]:
df.isna().sum()

,0
description,5
ingredients,0


In [ ]:
df[df['description'].isna()]

,description,ingredients
26084,None,"[{'count': '1 стак.', 'name': 'Рис'}, {'count'..."
39417,None,"[{'count': '1 шт', 'name': 'Папайя'}, {'count'..."
39436,None,"[{'count': '3 шт', 'name': 'Яйцо куриное'}, {'..."
39677,None,"[{'count': '250 г', 'name': 'Морковь'}, {'coun..."
39678,None,"[{'count': '2 шт', 'name': 'Язык свиной'}, {'c..."


In [ ]:
df.dropna(inplace=True)

Посмторим на длину текста описания и количество ингредиентов.

In [ ]:
df['len_description'] = df['description'].apply(lambda x: len(x.split()))
df['len_ingredients'] = df['ingredients'].apply(len)

In [ ]:
def plot_histogram(df, column_name):
    """
    Создает гистограмму для указанного столбца DataFrame с использованием Plotly.

    :param df: DataFrame, содержащий данные.
    :param column_name: Название столбца, для которого нужно построить гистограмму.
    """
    fig = px.histogram(df, x=column_name, title=f'Гистограмма для {column_name}')
    fig.update_layout(xaxis_title=column_name, yaxis_title='Частота')
    fig.show()

In [ ]:
plot_histogram(df, 'len_description')

Посмотрим на короткие тексты и очень длинные

In [ ]:
df[df['len_description'] <= 5]

,description,ingredients,len_description,len_ingredients
4,Фарш со вкусом виноградных листьев.,"[{'count': '500 г', 'name': 'Фарш мясной'}, {'...",5,7
5,Салат овощной,"[{'count': '500 г', 'name': 'Капуста квашеная'...",2,8
8,Картофельная запеканка с куриными грудками.,"[{'count': '1 шт', 'name': 'Грудка куриная'}, ...",5,6
10,Картофельная запеканка с куриной печенкой.,"[{'count': '1.5 кг', 'name': 'Картофель'}, {'c...",5,7
11,Кексы в формочках с изюмом,"[{'count': None, 'name': 'Мука пшеничная'}, {'...",5,7
...,...,...,...,...
152907,Полента из кукурузной крупы по-неаполитански,"[{'count': '300 г', 'name': 'Полента кукурузна...",5,15
152910,Гирос с чечевично-злаковой полентой BRAVOLLI.,"[{'count': '300 г', 'name': 'Полента чечевично...",5,14
152940,Мясной пирог из поленты BRAVOLLI,"[{'count': '200 г', 'name': 'Полента кукурузна...",5,6
152988,"Сытный, пикантный и вкусный салат.","[{'count': '100 г', 'name': 'Ветчина'}, {'coun...",5,6


In [ ]:
df[df['len_description'] >= 100]

,description,ingredients,len_description,len_ingredients
221,"""Павлова""- традиционный австралийский и новозе...","[{'count': '1 ст. л.', 'name': 'Сахарная пудра...",133,9
3396,Чернослив - хорошая штука! Научные исследовани...,"[{'count': '3 шт', 'name': 'Яйцо куриное'}, {'...",113,10
6725,С историей этих котлет связаны разные легенды....,"[{'count': '600 г', 'name': 'Филе куриное'}, {...",131,11
8074,В эти выходные хочу пригласить Вас в небольшое...,"[{'count': '1 шт', 'name': 'Рыба'}, {'count': ...",110,8
8833,"Этот рецептик ""родился"" в результате экспериме...","[{'count': '2 ', 'name': 'Телятина'}, {'count'...",102,9
...,...,...,...,...
153787,"До главного праздника года, осталось совсем не...","[{'count': '1 шт', 'name': 'Картофель'}, {'cou...",100,9
154014,"Новый год - время чудес! Да, я не ошиблась, эт...","[{'count': '500 мл', 'name': 'Молоко'}, {'coun...",128,9
154061,Яйцо на завтрак - это классика. А пицца? Скажи...,"[{'count': None, 'name': 'Мука пшеничная'}, {'...",100,9
154088,Крокембуш (croquembouche) - восхитительный фра...,"[{'count': '85 г', 'name': 'Сахар'}, {'count':...",100,22


Видим, что короткие тексты не очень информативны.

Длинные хоть и достаточно информативны, но также удалим для того, чтобы не было проблемы вида: маленький запрос или наооборот слишком большой.

In [ ]:
print(df.shape)
df = df[(df['len_description'] < 100) & (df['len_description'] > 5)]
df.shape

(154153, 4)


(141471, 4)

In [ ]:
plot_histogram(df, 'len_ingredients')

In [ ]:
df = df[df['len_ingredients'] < 23]
df.shape

(141037, 4)

Достанем из данных ```ingredients``` сами ингредиенты:

In [ ]:
df.iloc[3].values[1]

array([{'count': '1 шт', 'name': 'Щука'},
       {'count': '2 шт', 'name': 'Батон'},
       {'count': '1 шт', 'name': 'Яйцо куриное'},
       {'count': '50 г', 'name': 'Масло сливочное'},
       {'count': '1/4 стак.', 'name': 'Молоко'},
       {'count': '1 шт', 'name': 'Лук репчатый'},
       {'count': None, 'name': 'Соль'},
       {'count': None, 'name': 'Перец черный'},
       {'count': '6 шт', 'name': 'Лист лавровый'},
       {'count': '1/4 шт', 'name': 'Лимон'}], dtype=object)

In [ ]:
for product in df.iloc[3].values[1]:
  print(product['name'])

Щука
Батон
Яйцо куриное
Масло сливочное
Молоко
Лук репчатый
Соль
Перец черный
Лист лавровый
Лимон


In [ ]:
def get_ingredients(array):
    """
    Извлекает названия ингредиентов из массива продуктов и возвращает их в виде
     строки.
    Args:
        array (list of dict): Список продуктов, где каждый продукт представлен
        словарем
    Return:
        str: Строка, содержащая все названия ингредиентов, разделенные запятыми.

    """

    l = []
    for product in array:
        l.append(product['name'])
    return ', '.join(l)


morph = pymorphy3.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

def preprocess_description_text(text):
    """
    Предобрабатывает текст описания, удаляя цифры, знаки препинания, и приводит слова
    к нормальной форме, удаляет стоп-слова.

    Args:
        text (str): Текст для обработки
    Return:
        str: Обработанный текст, состоящий из нормализованных слов, без стоп-слов
        и знаков препинания.
    """
    text = re.sub(r'[^\w\s]|[\d]', '', str(text).lower())
    words = text.split()
    words = [morph.parse(word)[0].normal_form for word in words if word not in stop_words]

    return ' '.join(words)

In [ ]:
get_ingredients(df.iloc[3].values[1])

'Щука, Батон, Яйцо куриное, Масло сливочное, Молоко, Лук репчатый, Соль, Перец черный, Лист лавровый, Лимон'

In [ ]:
from tqdm import tqdm
tqdm.pandas()

df['description'] = df['description'].progress_apply(preprocess_description_text)
df['ingredients'] = df['ingredients'].progress_apply(lambda x: preprocess_description_text(get_ingredients(x)))

100%|██████████| 141037/141037 [04:02<00:00, 581.44it/s]


In [ ]:
# сохраним обработанные данные
df.to_csv('texts.csv', index=False)

In [ ]:
df = pd.read_csv('/content/texts.csv')[['description', 'ingredients']]
df.head()

,description,ingredients
0,украсить обычный бутерброд красный икра,батон икра красный маслина лимон зелень масло ...
1,обжарить баклажан жареный гриб сметанный соус,баклажан гриб лук репчатый сметана мука пшенич...
2,отбить кусочек мясо обжарить панировочный сухарь,свинина сухарь панировочный яйцо куриный соль ...
3,вообще это банкетный блюдо думать подать вечер...,щука батон яйцо куриный масло сливочный молоко...
4,оригинальный заливное форма яйцо стать прекрас...,яйцо куриный мясо колбаса желатина перец сладк...


In [ ]:
# def limit_words(text, max_words=45):
#     words = text.split()
#     if len(words) > max_words:
#         return ' '.join(words[:max_words])
#     return text

# # Примените эту функцию к колонке 'description'
# df['description'] = df['description'].apply(limit_words)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.dropna(inplace=True)

In [ ]:
train = df[:int(len(df)*0.9)]
test = df[int(len(df)*0.9):]
print(test.shape, train.shape)

(14103, 2) (126926, 2)


Используем TF-IDF для преобразования текстовых данных в числовые векторы

In [ ]:
vectorizer = TfidfVectorizer()

train['combined_text'] = train['description'] + " " + train['ingredients']
train_vectors = vectorizer.fit_transform(train['combined_text'])

<ipython-input-575-607395adac2b>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Рекомендации будут основываться на косинусной близости между векторыми представленичми:

$$\cos(\theta) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|}$$


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def recomendation_tf_idf(description: str, ingredients: str):
    """
    Функция для получения рекомендованных индексов на основе косинуса.
    Args:
        description (str): Описание продукта для поиска рекомендаций.
        ingredients (str): Состав продукта для поиска рекомендаций.
    Return:
        numpy.ndarray: Индексы 10 самых похожих продуктов из обучающего набора,
        отсортированные по убыванию сходства.
    """

    text_vector = vectorizer.transform([description + " " + ingredients])

    rating = cosine_similarity(train_vectors, text_vector).reshape(-1)
    rating_arg = np.argsort(rating)[::-1]

    return rating_arg[:5]

In [ ]:
rating = recomendation_tf_idf(test.iloc[36].values[0], test.iloc[36].values[1])

In [ ]:
test.iloc[36].values

array(['дорогой поварёнок хотеть предложить рецепт действительно сам быстрый пицца холодильник сайт рецепт пицца лаваш выпекаться духовка это лишний телодвижение поверьте результат сковородка ничуть плохой начинка абсолютно любой достаточно иметь главный ингредиент лаваш соус кетчуп сыр',
       'лаваш сыр полутвёрдый кетчуп салями шампиньон специя'],
      dtype=object)

In [ ]:
result = df.iloc[rating].iloc[1:]
print(result.shape)
result

(4, 2)


,description,ingredients
81501,вкусный чипсы лаваш микроволновка,лаваш сыр голландский кетчуп майонез смесь спе...
32840,очень вкусно лаваш получаться сочный сухой пов...,бедро куриный лаваш картофель майонез чеснок с...
47931,любить лаваш легко быстро главное вкусно,перец болгарский морковь тунец сыр плавленый о...
57142,быстрый вкусный рулет лаваш,лаваш морковь сыр твёрдый буженина майонез


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def compare_query_with_results(result_df, description, ingredients):
    """
    Вычисляет среднее косинусное сходство между запросом и готовой выдачей.

    :param result_df: DataFrame с колонками 'description' и 'ingredients'
    :param description: Описание рецепта
    :param ingredients: Ингредиенты
    :return: Список расстояний
    """
    result_df['combined_text'] = result_df['description'] + " " + result_df['ingredients']
    query_text = description + " " + ingredients

    # vectorizer = TfidfVectorizer()
    result_vectors = vectorizer.transform(result_df['combined_text'])

    query_vector = vectorizer.transform([query_text])

    similarities = cosine_similarity(result_vectors, query_vector).reshape(-1)
    mean_similarity = np.mean(similarities)

    return similarities

mean_sim = compare_query_with_results(result, test.iloc[36].values[0], test.iloc[36].values[1])

In [ ]:
result['cosine'] = mean_sim

In [ ]:
result

,description,ingredients,combined_text,cosine
81501,вкусный чипсы лаваш микроволновка,лаваш сыр голландский кетчуп майонез смесь спе...,вкусный чипсы лаваш микроволновка лаваш сыр го...,0.361488
32840,очень вкусно лаваш получаться сочный сухой пов...,бедро куриный лаваш картофель майонез чеснок с...,очень вкусно лаваш получаться сочный сухой пов...,0.349411
47931,любить лаваш легко быстро главное вкусно,перец болгарский морковь тунец сыр плавленый о...,любить лаваш легко быстро главное вкусно перец...,0.347034
57142,быстрый вкусный рулет лаваш,лаваш морковь сыр твёрдый буженина майонез,быстрый вкусный рулет лаваш лаваш морковь сыр ...,0.346585


Визуально рецепты действительно релевантные, несмотря на достаточно невысокое значение косинусной близости.

+ также учитываем, что запрос брали из теста, который не пересекается с тренировочной частью

In [ ]:
result['cosine'].mean()

0.3511296441551313

In [ ]:
#подбор гиперпараметров веторизатора качество не улучшил, ngram_range даже сильно ухудшил
param_grid = {
    'max_df': np.arange(0.5, 1.1, 0.1),
    'min_df': np.arange(0, 1.0, 0.1),
    'ngram_range': [(1, 1), (1, 2)],
    'max_features': np.arange(1000, 5000, 100),
}

In [ ]:
def pipeline_tfidf():
    """
    Выполняет пайплайн для создания рекомендаций с использованием TF-IDF
    векторизации и вычисления сходства косинуса.

    Return:
        list: Список средних значений сходства косинуса для каждого тестового
        примера.
    """
    score = []
    vectorizer = TfidfVectorizer()

    train['combined_text'] = train['description'] + " " + train['ingredients']
    train_vectors = vectorizer.fit_transform(train['combined_text'])

    def process_row(rows):
        rating = recomendation_tf_idf(rows['description'], rows['ingredients'])
        result = df.iloc[rating]
        mean_sim = compare_query_with_results(result, rows['description'], rows['ingredients'])
        result.loc[:, 'cosine'] = mean_sim
        return result['cosine'].mean()

    score = test.progress_apply(process_row, axis=1)

    return score.tolist()

In [ ]:
score = pipeline_tfidf()

<ipython-input-480-6ff970092b84>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

100%|██████████| 14103/14103 [18:46<00:00, 12.52it/s]


In [ ]:
test['scores'] = score
test

<ipython-input-587-2e4e05f82386>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,description,ingredients,scores
126934,вкусный простой нежный пирог сухой влажный тек...,яйцо куриный сахар мука пшеничный кефир разрых...,0.351985
126935,пышный воздушный пампушка чеснок борщ рецепт ч...,мука пшеничный дрожжи сахар вода соль масло ра...,0.611697
126936,готовить вкусный простой пирог яблоко хотеться...,яйцо куриный масло сливочный соль сахар мука п...,0.396861
126937,сербия вариация свёрнутый рогалик запечь тест ...,мука пшеничный дрожжи соль молоко масло сливоч...,0.300093
126938,сегодня хотеть поделиться рецепт потрясать неж...,мука пшеничный вода дрожжи масло растительный ...,0.389689
...,...,...,...
141032,нежный романтический десерт черника бисквитный...,бисквит черника сахар желатина вода сыр творож...,0.477537
141033,очень вкусный картофель новый исполнение,мука пшеничный картофель сметана яйцо куриный ...,0.448225
141034,день второй салат капустный морковь свёкла лук...,капуста белокочанный морковь свёкла лук репчат...,0.429102
141035,любитель морской рыба запечь духовка овощной г...,окунь морской шампиньон чеснок масло сливочный...,0.423309


In [ ]:
np.mean(score)

0.38286991691080496

Вывод:

Рекомендательная система рецептов действительно работает, визуально мы получаем схожие рецепты.

Косинусное сходство в среднем достаточно небольшое (хотя есть и высокие значения), это связано с большим разбросом в длине рецептов и количестве неиформативных слов. Так как используется TF-IDF, который не учитывает контекст.

